In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from copy import deepcopy
import os
import os.path as op
import sys
from matplotlib import pyplot as plt
import statsmodels as sm
import statsmodels.api as sma

loc = 'workstation'
if loc == 'laptop':
    #eyefuncdir = '/Users/sammichekroud/Desktop/postdoc/student_projects/EffortDifficulty/analysis/tools'
    eyefuncdir = '/Users/sammichekroud/Desktop/postdoc/tools'
    wd         = '/Users/sammichekroud/Desktop/postdoc/wmconfidence' #working on confidence data, but in postdoc dir
elif loc == 'workstation':
    eyefuncdir = 'C:/Users/sammirc/Desktop/postdoc/tools/'
    wd         =  'C:/Users/sammirc/Desktop/postdoc/tuningcurves'
    funcdir    = op.join(wd, 'analysis', 'tools')
    sys.path.insert(0, funcdir)
os.chdir(wd)
# sys.path.insert(0, eyefuncdir)
sys.path.insert(0, op.join(wd, 'analysis', 'tools'))

from funcs import getSubjectInfo, clusterperm_test

eyedir = op.join(wd, 'data', 'eyes')
bdir   = op.join(wd, 'data', 'datafiles')

subs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26])
subs = np.array([         4, 5, 6, 7, 8, 9,     11, 12, 13, 14, 15, 16, 17, 18,     20, 21, 22,     24, 25, 26]) #eeg ppts
# subs = np.array([         4, 5, 6, 7, 8, 9,             13, 14, 15,     17,         20, 21, 22,     24, 25, 26]) # eyetracking ppts
nsubs = subs.size
#set some params here
modeltimes = np.round(np.load(op.join(wd, 'data', 'tuningcurves', 'times.npy')), 2)
ntimes = modeltimes.size

#read in the modelled data
#save across subject betas in one file
binstep, binwidth, weightTrials = 4, 22, True
# binstep, binwidth, weightTrials = 15, 22, True

b_prec = np.load(op.join(wd, 'data', 'glms', 'tc_behaviour', 'glm1', f'TCParamsXbehaviour_beta_precision_binstep{binstep}_binwidth{binwidth}.npy'))
b_amp  = np.load(op.join(wd, 'data', 'glms', 'tc_behaviour', 'glm1', f'TCParamsXbehaviour_beta_amplitude_binstep{binstep}_binwidth{binwidth}.npy'))
t_prec = np.load(op.join(wd, 'data', 'glms', 'tc_behaviour', 'glm1', f'TCParamsXbehaviour_tstat_precision_binstep{binstep}_binwidth{binwidth}.npy'))
t_amp  = np.load(op.join(wd, 'data', 'glms', 'tc_behaviour', 'glm1', f'TCParamsXbehaviour_tstat_amplitude_binstep{binstep}_binwidth{binwidth}.npy'))

b_precd = np.load(op.join(wd, 'data', 'glms', 'tc_behaviour', 'glm1', 'firstderiv_TCParamsXbehaviour_beta_precision.npy'))
b_ampd  = np.load(op.join(wd, 'data', 'glms', 'tc_behaviour', 'glm1', 'firstderiv_TCParamsXbehaviour_beta_amplitude.npy'))
t_precd = np.load(op.join(wd, 'data', 'glms', 'tc_behaviour', 'glm1', 'firstderiv_TCParamsXbehaviour_tstat_precision.npy'))
t_ampd  = np.load(op.join(wd, 'data', 'glms', 'tc_behaviour', 'glm1', 'firstderiv_TCParamsXbehaviour_tstat_amplitude.npy'))

smoothamp = False
if smoothamp:
    #if you want to smooth single-participant time series
    b_amp, t_amp = sp.ndimage.gaussian_filter1d(b_amp, sigma=2), sp.ndimage.gaussian_filter1d(t_amp, sigma=2)

#for completeness in the script, we also want to get just the across participant time series for tuning curve precision and amplitude
nitems, nparams = 2, 2
subcount = -1
precision = np.zeros(shape = [nsubs, ntimes]) * np.nan
amplitude = np.zeros(shape = [nsubs, nparams, ntimes]) * np.nan
for i in subs:
    subcount +=1
    #print(f'working on ppt {subcount+1}/{subs.size}')
    iprec = np.load(op.join(wd, 'data', 'tuningcurves', 'parameter_fits', 'twostage_alphaminmaxfit_b1desmatminmax',
                f's{i}_ParamFits_precision_binstep{binstep}_binwidth{binwidth}_smoothedprec.npy'))
    
    iamp = np.load(op.join(wd, 'data', 'tuningcurves', 'parameter_fits', 'twostage_alphaminmaxfit_b1desmatminmax',
            f's{i}_ParamFits_amplitude_binstep{binstep}_binwidth{binwidth}_smoothedprec_glmfit.npy'))
    
    precision[subcount] = np.nanmean(iprec, axis=1).mean(0) #average across trials, then average across items
    amplitude[subcount] = np.nanmean(iamp, axis=1).mean(0) #average across trials, then average across items
print('done')

done


In [2]:
gm_prec = precision.mean(0); sem_prec = sp.stats.sem(precision, axis=0, ddof=0, nan_policy='omit')
gm_amp  = amplitude.mean(0); sem_amp = sp.stats.sem(amplitude, axis=0, ddof=0, nan_policy='omit')

#get mean/sem timeseries for relationship between tuning curve parameters and behaviour
gm_bp = b_prec.mean(0); sem_bp = sp.stats.sem(b_prec, axis=0, ddof=0, nan_policy = 'omit')
gm_ba = b_amp.mean(0);  sem_ba = sp.stats.sem(b_amp, axis=0, ddof=0, nan_policy = 'omit')
gm_tp = t_prec.mean(0); sem_tp = sp.stats.sem(t_prec, axis=0, ddof=0, nan_policy = 'omit')
gm_ta = t_amp.mean(0);  sem_ta = sp.stats.sem(t_amp, axis=0, ddof=0, nan_policy = 'omit')

gm_bpd = b_precd.mean(0); sem_bpd = sp.stats.sem(b_precd, axis=0, ddof=0, nan_policy = 'omit')
gm_bad = b_ampd.mean(0);  sem_bad = sp.stats.sem(b_ampd, axis=0, ddof=0, nan_policy = 'omit')
gm_tpd = t_precd.mean(0); sem_tpd = sp.stats.sem(t_precd, axis=0, ddof=0, nan_policy = 'omit')
gm_tad = t_ampd.mean(0);  sem_tad = sp.stats.sem(t_ampd, axis=0, ddof=0, nan_policy = 'omit')

In [ ]:
np.random.seed(420)

sigalpha = 0.05
t_thresh = sp.stats.t.ppf(1-sigalpha, df = nsubs-1)
tmin, tmax = 0.0,  0.75 #time-window for cluster permutation testing
nperms = 'all'
nperms = 10000

# first, visualise precision
fig = plt.figure(figsize = [6,5])
ax1 = fig.add_subplot(311)
# ax1 = fig.add_subplot(111)
ax1.plot(modeltimes, gm_prec, lw = 1.5, color = '#e34a33', label = 'mean')
ax1.fill_between(modeltimes, np.add(gm_prec, sem_prec), np.subtract(gm_prec, sem_prec), lw = 0, edgecolor=None, alpha = 0.3, color = '#e34a33')
ax1.set_ylabel('tuning curve\nprecision (AU)')
ax1.set_title('timecourse of tuning curve precision across participants', fontsize=10)
ax2 = fig.add_subplot(312)
# ax2 = ax1.twinx()
ax2.plot(modeltimes, gm_bp[1], lw = 1.5, color = '#3182bd', label = 'error')
ax2.fill_between(modeltimes, np.add(gm_bp[1], sem_bp[1]), np.subtract(gm_bp[1], sem_bp[1]), lw = 0, edgecolor = None, alpha = 0.3, color = '#3182bd')
ax2.set_ylabel('beta (AU)')
ax3 = fig.add_subplot(313)
ax3.plot(modeltimes[1:], gm_bpd[1], lw = 1.5, color = '#4daf4a', label = '1st deriv error')
ax3.fill_between(modeltimes[1:], np.add(gm_bpd[1], sem_bpd[1]), np.subtract(gm_bpd[1], sem_bpd[1]), lw = 0, edgecolor = None, alpha = 0.3, color = '#4daf4a')

for ax in [ax1, ax2, ax3]:
    # ax.set_ylabel('$\\beta$')
    # ax.set_title(f'tuning curve precision')
    ax.axvline(0, ls = 'dashed', lw = 1, color='k')
    ax.set_xlim([-0.5, 1])
    ax.set_xticks(np.arange(-0.5, 1.5, 0.5))
    ax.tick_params(axis='x', which='major', labelsize=10)
    ax.set_xticks(np.arange(-0.5, 1.1, 0.1), minor=True)
ax2.axhline(0, ls = 'dashed', lw = 0.5, color = 'k')
ax3.axhline(0, ls = 'dashed', lw = 0.5, color = 'k')
ax3.set_xlabel('time relative to array onset (s)')
ax2.set_title('timecourse of relationship between tuning curve precision\nand single-trial WM error', fontsize=10)
ax3.set_title('timecourse of relationship between first derivative of\ntuning curve precision and WM error', fontsize=10) 

#cluster stats for timecourse of WM error ~ tuning curve precision 
glmlabs = np.array(['mean', 'precision'])
regname = 'precision'
tv, clu, clupv, _ = clusterperm_test(data = b_prec, labels = glmlabs, of_interest = regname, times = modeltimes,
                                     tmin = tmin, tmax = tmax, out_type='indices', n_permutations = nperms,
                                     threshold = -t_thresh, tail=-1, n_jobs=4) #one-tailed test
clu = [x[0] for x in clu]
print(f'cluster p-values for {regname} regressor: {clupv}')
times_twin = modeltimes[np.logical_and(modeltimes>=tmin, modeltimes <= tmax)]
nclus = len(clu)
for icluster in range(nclus):
    mask = clu[icluster]
    if clupv[icluster] <= sigalpha:
        itmin = times_twin[mask[0]]
        itmax = times_twin[mask[-1]]
        ax2.hlines(y = -0.04, xmin = itmin, xmax = itmax, lw = 3, color = '#3182bd', alpha = 1)

#cluster stats for timecourse of WM error ~ tuning curve precision 
glmlabs = np.array(['mean', 'precision'])
regname = 'precision'
tv, clu, clupv, _ = clusterperm_test(data = b_precd, labels = glmlabs, of_interest = regname, times = modeltimes,
                                     tmin = tmin, tmax = tmax, out_type='indices', n_permutations = nperms,
                                     threshold = t_thresh, tail=0, n_jobs=4) #one-tailed test
clu = [x[0] for x in clu]
print(f'cluster p-values for {regname} regressor: {clupv}')
times_twin = modeltimes[np.logical_and(modeltimes>=tmin, modeltimes <= tmax)]
nclus = len(clu)
for icluster in range(nclus):
    mask = clu[icluster]
    if clupv[icluster] <= sigalpha:
        itmin = times_twin[mask[0]]
        itmax = times_twin[mask[-1]]
        ax3.hlines(y = -0.027, xmin = itmin, xmax = itmax, lw = 3, color = '#4daf4a', alpha = 1)


fig.tight_layout()

stat_fun(H1): min=-2.8782720526420595 max=0.9847735796738897
Running initial clustering …
Found 2 clusters


C:\Users\sammirc\anaconda3\envs\mne\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| Permuting : 9999/9999 [00:03<00:00, 3022.38it/s]

cluster p-values for precision regressor: [0.0859 0.0287]
stat_fun(H1): min=-2.8887949721543236 max=3.5702430475536517
Running initial clustering …
Found 3 clusters



 26%|██▌       | Permuting : 2581/9999 [00:00<00:00, 13610.27it/s]

In [ ]:
np.random.seed(420)

#same process but for amplitude now instead
sigalpha = 0.05
df = nsubs-1
t_thresh = sp.stats.t.ppf(1-sigalpha, df = df)
tmin, tmax = 0.0,  0.75 #time-window for cluster permutation testing
nperms = 'all'
nperms = 10000

# first, visualise precision
fig = plt.figure(figsize = [6,5])
ax1 = fig.add_subplot(311)
# ax1 = fig.add_subplot(111)
ax1.plot(modeltimes, gm_amp[0], lw = 1.5, color = '#756bb1', label = 'mean')
ax1.fill_between(modeltimes, np.add(gm_amp[0], sem_amp[0]), np.subtract(gm_amp[00], sem_amp[0]), lw = 0, edgecolor=None, alpha = 0.3, color = '#756bb1')
ax1.set_ylabel('tuning curve\namplitude (AU)')
ax1.set_title('timecourse of tuning curve amplitude across participants', fontsize=10)
ax2 = fig.add_subplot(312)
# ax2 = ax1.twinx()
ax2.plot(modeltimes, gm_ba[1], lw = 1.5, color = '#3182bd', label = 'error')
ax2.fill_between(modeltimes, np.add(gm_ba[1], sem_ba[1]), np.subtract(gm_ba[1], sem_ba[1]), lw = 0, edgecolor = None, alpha = 0.3, color = '#3182bd')
ax2.set_ylabel('beta (AU)')
ax3 = fig.add_subplot(313)
ax3.plot(modeltimes[1:], gm_bad[1], lw = 1.5, color = '#4daf4a', label = '1st deriv error')
ax3.fill_between(modeltimes[1:], np.add(gm_bad[1], sem_bad[1]), np.subtract(gm_bad[1], sem_bad[1]), lw = 0, edgecolor = None, alpha = 0.3, color = '#4daf4a')

for ax in [ax1, ax2, ax3]:
    # ax.set_ylabel('$\\beta$')
    # ax.set_title(f'tuning curve precision')
    ax.axvline(0, ls = 'dashed', lw = 1, color='k')
    ax.set_xlim([-0.5, 1])
    ax.set_xticks(np.arange(-0.5, 1.5, 0.5))
    ax.tick_params(axis='x', which='major', labelsize=10)
    ax.set_xticks(np.arange(-0.5, 1.1, 0.1), minor=True)
ax2.axhline(0, ls = 'dashed', lw = 0.5, color = 'k')
ax3.axhline(0, ls = 'dashed', lw = 0.5, color = 'k')
ax3.set_xlabel('time relative to array onset (s)')
ax2.set_title('timecourse of relationship between tuning curve amplitude\nand single-trial WM error', fontsize=10)
ax3.set_title('timecourse of relationship between first derivative of\ntuning curve amplitude and WM error', fontsize=10) 

#cluster stats for timecourse of WM error ~ tuning curve precision 
glmlabs = np.array(['mean', 'amplitude'])
regname = 'amplitude'
tv, clu, clupv, _ = clusterperm_test(data = b_amp, labels = glmlabs, of_interest = regname, times = modeltimes,
                                     tmin = tmin, tmax = tmax, out_type='indices', n_permutations = nperms,
                                     threshold = -t_thresh, tail=-1, n_jobs=4) #one-tailed test
clu = [x[0] for x in clu]
print(f'cluster p-values for {regname} regressor: {clupv}')
times_twin = modeltimes[np.logical_and(modeltimes>=tmin, modeltimes <= tmax)]
nclus = len(clu)
for icluster in range(nclus):
    mask = clu[icluster]
    if clupv[icluster] <= sigalpha:
        itmin = times_twin[mask[0]]
        itmax = times_twin[mask[-1]]
        ax2.hlines(y = -0.04, xmin = itmin, xmax = itmax, lw = 3, color = '#3182bd', alpha = 1)

#cluster stats for timecourse of WM error ~ tuning curve precision 
glmlabs = np.array(['mean', 'amplitude'])
regname = 'amplitude'
tv, clu, clupv, _ = clusterperm_test(data = b_ampd, labels = glmlabs, of_interest = regname, times = modeltimes,
                                     tmin = tmin, tmax = tmax, out_type='indices', n_permutations = nperms,
                                     threshold = t_thresh, tail=0, n_jobs=4) #one-tailed test
clu = [x[0] for x in clu]
print(f'cluster p-values for {regname} regressor: {clupv}')
times_twin = modeltimes[np.logical_and(modeltimes>=tmin, modeltimes <= tmax)]
nclus = len(clu)
for icluster in range(nclus):
    mask = clu[icluster]
    if clupv[icluster] <= sigalpha:
        itmin = times_twin[mask[0]]
        itmax = times_twin[mask[-1]]
        ax3.hlines(y = -0.027, xmin = itmin, xmax = itmax, lw = 3, color = '#4daf4a', alpha = 1)


fig.tight_layout()